In [63]:
# 앙상블의 유형은 일반적으로 보팅 배깅 부스팅으로 구분할수있다
# 뛰어난 성능을 가진 모델들로만 구성하는 것보다 성능이 떨어지더라도
# 서로 다른 유형의 모델을 섞는 것이 오히려 전체 성능에 도움이 된다
# 부스팅은
# 보팅 여러개의 학습지를 사용하는것
# 하드보팅 다수결 // 소프트보팅 클래스별 평균

In [107]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# ensemble 위한 분류학습기 추가하기

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.preprocessing import LabelEncoder , OneHotEncoder , StandardScaler , MinMaxScaler , Binarizer 
from sklearn.model_selection import train_test_split , GridSearchCV

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score , roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_curve , roc_curve

import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns

import missingno as ms
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

from sklearn.datasets import load_breast_cancer

### RandomForest (Bagging)

In [65]:
# 배깅은 보팅과는 달리 동일한 알고리즘으로 여러 분류기를 만들어서 
# 보팅으로 최종결정하는 알고리즘

# 부트스트래핑은 전체 데이터에서 일부 데이터의 중첩을 허용하는 방식(복원추출)

- 배깅 장점
- 앙상블 방식의 알고리즘 중 수행속도가 빠르다
- 다양한 데이터세트에서의 좋은 성능

- 단점
- 튜닝을 위한 시간이 오래걸린다 

In [66]:
# HAR 데이터를 이용해서 RandomForest 배깅을 이용한 분류예측

In [67]:
# 각 데이터 파일들은 공백으로 분리되어 있으므로 read_csv에서 공백문자를 sep으로 할당
feature_name_df = pd.read_csv('./data/features.txt', sep='\s+',header=None, names=['column_index', 'column_name'])

In [68]:
# 데이터프레임에 피처명을 컬럼으로 뷰여하기 위해 리스트 객체로 다시 반환
feature_name = feature_name_df.iloc[: , 1].values.tolist()

In [69]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0]+'_'+str(x[1]) 
                                                                                           if x[1] >0 else x[0] ,  axis=1)
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [70]:
new_feature_name_df=get_new_feature_name_df(feature_name_df)
new_feature_name_df.head()

,column_index,column_name,dup_cnt
0,1,tBodyAcc-mean()-X,0
1,2,tBodyAcc-mean()-Y,0
2,3,tBodyAcc-mean()-Z,0
3,4,tBodyAcc-std()-X,0
4,5,tBodyAcc-std()-Y,0


In [71]:
feature_name = new_feature_name_df.iloc[: , 1].values.tolist()

In [72]:
X_train = pd.read_csv('./data/train/X_train.txt', sep='\s+', header=None,names=feature_name)
print( type(X_train))
X_test = pd.read_csv('./data/test/X_test.txt', sep='\s+', header=None,names=feature_name)
    
# 학습 레이블과 테스트 레이블 데이터를 데이터 프레임으로 로딩, 컬럼명은 action으로 부여
y_train = pd.read_csv('./data/train/y_train.txt', sep='\s+', names=['action'])
y_test = pd.read_csv('./data/test/y_test.txt', sep='\s+', names=['action'])

<class 'pandas.core.frame.DataFrame'>


In [73]:
# 랜덤포레스트 알고리즘 이용한 학습/예측/평가
rf_model=RandomForestClassifier(random_state=0)
rf_model.fit(X_train,y_train) #학습
y_pred=rf_model.predict(X_test)

In [74]:
print('answer',y_test)
print('guess',y_pred)
accuracy=accuracy_score(y_test,y_pred)
print('RF ACCURACY :',accuracy)

answer       action
0          5
1          5
2          5
3          5
4          5
...      ...
3157       2
3158       2
3159       2
3160       2
3161       2

[3162 rows x 1 columns]
guess [5 5 5 ... 2 2 1]
RF ACCURACY : 0.9108159392789373


In [75]:
# GridSeatchCV 교차검증 및 하이퍼 파라미터 튜닝

# 파라미터 튜닝을 위해서  params을 만들어야함
params = {
    'n_estimators' : [100],
    'max_depth' : [6,8,10,12],
    'min_samples_leaf' :[8,12,18],
    'min_samples_split' : [8,16,20]
}

cv_rf_model = RandomForestClassifier(random_state=0,n_jobs=-1)
grid_cv = GridSearchCV(cv_rf_model, param_grid=params,cv=5,n_jobs=-1)
grid_cv.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid={'max_depth': [6, 8, 10, 12],
                         'min_samples_leaf': [8, 12, 18],
                         'min_samples_split': [8, 16, 20],
                         'n_estimators': [100]})

In [76]:
print('최적의 파라미터:',grid_cv.best_params_)
print('예측 정확도:',grid_cv.best_score_)

최적의 파라미터: {'max_depth': 12, 'min_samples_leaf': 12, 'min_samples_split': 8, 'n_estimators': 100}
예측 정확도: 0.9115538406587987


In [101]:
# 최적의 하이퍼 피라미터를 이용하여 랜덤 포레스트를 다시 학습시켜보세요
hyper_gbc_model=RandomForestClassifier(max_depth=12,
                                       min_samples_leaf=12,
                                       min_samples_split=8,
                                       n_estimators=100,
                                       random_state=0,
                                       n_jobs=-1)
hyper_gbc_model.fit(X_train,y_train)
hyper_y_pred=hyper_rf_model.predict(X_test)

In [102]:
print('튜닝을 통한 예측 정확도:',accuracy_score(y_test,hyper_y_pred))

튜닝을 통한 예측 정확도: 0.9041745730550285


In [108]:
# 피처임포턴트 시각화 해보세요!
ftr_importances_values=hyper_gbc_model.feature_importances_
ftr_importances=pd.Series(ftr_importances_values,index=X_train.columns)
ftr_top20=ftr_importances.sort_values(ascending=False)[:20]
print(ftr_top20)

angle(X,gravityMean)              0.038086
tGravityAcc-mean()-Y              0.032561
tGravityAcc-min()-X               0.032025
tGravityAcc-max()-X               0.026187
tGravityAcc-max()-Y               0.025790
angle(Y,gravityMean)              0.024956
tGravityAcc-mean()-X              0.024410
tGravityAcc-energy()-X            0.023287
tGravityAcc-min()-Y               0.022826
tGravityAcc-energy()-Y            0.015522
tBodyAcc-max()-X                  0.013679
fBodyAcc-energy()-X               0.012200
fBodyAccJerk-bandsEnergy()-1,8    0.011923
tGravityAcc-max()-Z               0.011163
tGravityAcc-arCoeff()-Z,1         0.011096
tBodyAccJerk-iqr()-Y              0.010639
tGravityAcc-min()-Z               0.010307
tGravityAcc-arCoeff()-Z,2         0.010232
fBodyAccJerk-mean()-X             0.009778
tGravityAccMag-std()              0.009731
dtype: float64
